<a href="https://colab.research.google.com/github/nkhllg/Scriptie/blob/main/scriptie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers sentence-transformers torch
!pip install faiss-cpu
!pip install huggingface_hub[hf_xet]
!pip install sentence-transformers faiss-cpu transformers torch
!pip install geopy


In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import T5Tokenizer, T5ForConditionalGeneration
import faiss
import numpy as np


# -------------------------------
# 1. Load models
# -------------------------------
embedder = SentenceTransformer('sentence-t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')
generator = T5ForConditionalGeneration.from_pretrained('t5-base')

# -------------------------------
# 2. Embed your corpus
# -------------------------------
documents = [
    "Python is a programming language.",
    "Paris is the capital of France.",
    "T5 is a text-to-text transformer model developed by Google.",
    "FAISS is used for efficient similarity search on embeddings.",
    "The company announced a new product launch scheduled for next month.",
    "The researchers have discovered a new method to improve data processing efficiency.",
"The team completed the project ahead of the original deadline.",
"The new software update includes several performance enhancements and bug fixes."
]

print('docs read')

doc_embeddings = embedder.encode(documents, convert_to_numpy=True)

print('embedded')
# -------------------------------
# 3. Create a FAISS index
# -------------------------------
embedding_dim = doc_embeddings.shape[1]
print('embedding')
index = faiss.IndexFlatL2(embedding_dim)
print('index')
index.add(doc_embeddings)

# -------------------------------
# 4. Query with similarity search
# -------------------------------
query = "when is the product launch scheduled?"
query_embedding = embedder.encode(query, convert_to_numpy=True)

print('query embedding')

k = 2
distances, indices = index.search(np.array([query_embedding]), k)
print('distance')
retrieved_docs = [documents[i] for i in indices[0]]

print('retrieved')
# -------------------------------
# 5. Format input for T5 generation
# -------------------------------
context = " ".join(retrieved_docs)
input_text = f"question: {query} context: {context}"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# -------------------------------
# 6. Generate answer using T5
# -------------------------------
output_ids = generator.generate(input_ids, max_length=50)
answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Query:", query)
print("Answer:", answer)


docs read
embedded
embedding
index
query embedding
distance
retrieved
Query: when is the product launch scheduled?
Answer: next month
